In [1]:
import pandas as pd
import json
import requests
import os
from tqdm import tqdm

In [2]:
load_info_path = 'filereport_read_run_PRJEB51982_json.txt'

with open(load_info_path) as f:
    data = json.load(f)

data_dir = 'data'

In [3]:
# load done link
done_link = set()

if os.path.exists('done_link.txt'):
    with open('done_link.txt') as f:
        for line in f:
            done_link.add(line.strip())

In [4]:
def download_fastq(ftp_link, dest_folder):
    """Download FASTQ files from the given FTP link to the specified destination folder."""
    # Convert FTP to HTTP for ENA's server
    http_url = "http://" + ftp_link
    filename = http_url.split("/")[-1]
    dest_path = os.path.join(dest_folder, filename)
    
    response = requests.get(http_url, stream=True)
    with open(dest_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

In [5]:
len(done_link)

0

In [6]:
for record in tqdm(data, desc="Downloading FASTQ files"):
    sample_id = record['sample_accession']
    fastq_links = record['fastq_ftp'].split(";")
    
    # Create a directory for the sample if it doesn't exist
    sample_dir = os.path.join(data_dir, sample_id)
    if not os.path.exists(sample_dir):
        os.mkdir(sample_dir)

    # Download each FASTQ file to the sample's directory
    for link in fastq_links:
        if link in done_link:
            continue

        download_fastq(link, sample_dir)
        done_link.add(link)

In [8]:
# save done link
with open('done_link.txt', 'w') as f:
    for item in done_link:
        f.write("%s\n" % item)